In [1]:
from math import radians

import numpy as np
import plotly.graph_objects as go
import pandas as pd
import folium
import pvlib
from pvlib import solarposition


In [2]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [26]:
year_range = pd.date_range("01/01/2020 00:00:00", "31/12/2020 00:00:00", freq="H", tz="Europe/Oslo")
lat, lon = 59.946247,10.761360
data = solarposition.get_solarposition(year_range,lat, lon)

In [4]:
def create_year_data(base_year=2020):
    year_range = pd.date_range(f"01/01/{base_year} 00:00:00", f"31/12/{base_year} 00:00:00", freq="H", tz="Europe/Oslo")
    lat, lon = 59.946247,10.761360
    data = solarposition.get_solarposition(year_range,lat, lon)
    return data

In [27]:
def create_today_data(date="2020.02.05", lat=59.946247, lon=10.761360, c=0.001):
    data = create_year_data(date.split(".")[0])
    sun_hours = data[data["elevation"] > 0]
    today_data = sun_hours.loc[date]
    lon_factor = 1/np.cos(np.radians(lat))
    today_data["lat"] = lat + c*np.cos(np.radians(today_data["azimuth"]))*np.cos(np.radians(today_data["elevation"]))
    today_data["lon"] = lon + c*lon_factor*np.sin(np.radians(today_data["azimuth"]))*np.cos(np.radians(today_data["elevation"]))

    return today_data
    

In [47]:
def create_map(date="2020.02.05", lat=59.946247, lon=10.761360):
    today_data = create_today_data(date=date)
    hours = [str(hour) for hour in today_data.index.hour]
    mapbox_access_token = open("mapbox_token.txt").read()

    fig = go.Figure(go.Scattermapbox(
            lat=today_data["lat"],
            lon=today_data["lon"],
            mode='markers+text',
            marker=go.scattermapbox.Marker(
                size=22,
                color="orange"
            ),
            text=hours,
            name = "Hour of the day",
            textfont=dict(
            family="sans serif",
            size=18,
            color="black"
            )
        ))

    fig.add_trace(go.Scattermapbox(
            lat=[lat],
            lon=[lon],
            mode='markers+text',
            name = "Position",
            marker=go.scattermapbox.Marker(
                size=14,
                color="blue"
            )
        ))

    fig.update_layout(
        autosize=True,
        hovermode='closest',
        height=700,
        mapbox=go.layout.Mapbox(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=lat,
                lon=lon
            ),
            pitch=0,
            zoom=16
        ),
    )

    fig.show()

In [49]:
create_map(date="2020.08.27")

In [ ]:
mapbox_access_token = open("mapbox_token.txt").read()

fig = go.Figure(go.Scattermapbox(
        lat=today_data["lat"],
        lon=today_data["lon"],
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=22,
            color="orange"
        ),
        text=hours,
        textfont=dict(
        family="sans serif",
        size=18,
        color="black"
    )
    ))

fig.add_trace(go.Scattermapbox(
        lat=[lat],
        lon=[lon],
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=22,
            color="black"
        )
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    height=700,
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=lat,
            lon=lon
        ),
        pitch=0,
        zoom=17
    ),
)

fig.show()